<a href="https://colab.research.google.com/github/Xziipx/Projects/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


train_df = pd.read_csv('/content/drive/MyDrive/data bigdata/train_df (1).csv')
test_df = pd.read_csv('/content/drive/MyDrive/data bigdata/test_df (1).csv')
y_train = pd.read_csv('/content/drive/MyDrive/data bigdata/y_train.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train) #Transforming the target variable using labels
#We see that the destination countries have been successfully encoded now
encoded_y_train

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 7,  7, 10, ...,  7,  7,  7])

In [ ]:
print(encoded_y_train.shape)
print(train_df.shape)

(213451,)
(213451, 1280)


##Model 1

In [ ]:
  from sklearn.preprocessing import LabelBinarizer
  def dcg_score(y_true, y_score, k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    '''
        Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.
    '''

    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score_udf(ground_truth, predictions, k=5):
    lb = LabelBinarizer()
    lb.fit(range(predictions.shape[1] + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

In [ ]:
#Building model 1
from sklearn.ensemble import RandomForestClassifier
n_estimator= [5,10,50,100,500]
max_depth= [3,4,6]

for i in max_depth:
    for j in (n_estimator):
        clf_rf = RandomForestClassifier(n_estimators=j, criterion='gini', max_depth=i, random_state=42, n_jobs=-1)
        model1 = clf_rf.fit(train_df,encoded_y_train)
        best2_score = ndcg_score_udf(encoded_y_train, model1.predict_proba(train_df))
        print(f"NDCG Training Score on the first best estimator: {best2_score}")
        print ("Depth of tree : {}".format(i))
        print ("Number of Estimators : {}".format(j))
        print("*****************")

NDCG Training Score on the first best estimator: 0.8141053504160195
Depth of tree : 3
Number of Estimators : 5
*****************
NDCG Training Score on the first best estimator: 0.8139917599224357
Depth of tree : 3
Number of Estimators : 10
*****************
NDCG Training Score on the first best estimator: 0.8141185341034587
Depth of tree : 3
Number of Estimators : 50
*****************
NDCG Training Score on the first best estimator: 0.8140435625308199
Depth of tree : 3
Number of Estimators : 100
*****************
NDCG Training Score on the first best estimator: 0.8140447976629298
Depth of tree : 3
Number of Estimators : 500
*****************
NDCG Training Score on the first best estimator: 0.8145037602981511
Depth of tree : 4
Number of Estimators : 5
*****************
NDCG Training Score on the first best estimator: 0.8146056143289129
Depth of tree : 4
Number of Estimators : 10
*****************
NDCG Training Score on the first best estimator: 0.8144618430855458
Depth of tree : 4
Numb

In [ ]:
#Best params in Random Forest  is depth = 6 and number of estimators = 500
clf_rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=6, random_state=42, n_jobs=-1)
model1 = clf_rf.fit(train_df,encoded_y_train)
best2_score = ndcg_score_udf(encoded_y_train, model1.predict_proba(train_df))
print(f"NDCG Training Score on the first best estimator: {best2_score}")

NDCG Training Score on the first best estimator: 0.8152533605498138


In [ ]:
test_df

,age,day_account_created,month_account_created,year_account_created,day_first_active,month_first_active,year_first_active,time_lag,null_x,10_x,...,weekday_account_created_Thursday,weekday_account_created_Tuesday,weekday_account_created_Wednesday,weekday_first_active_Friday,weekday_first_active_Monday,weekday_first_active_Saturday,weekday_first_active_Sunday,weekday_first_active_Thursday,weekday_first_active_Tuesday,weekday_first_active_Wednesday
0,35.0,1,7,2014,1,7,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
1,-1.0,1,7,2014,1,7,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
2,-1.0,1,7,2014,1,7,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
3,-1.0,1,7,2014,1,7,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
4,-1.0,1,7,2014,1,7,2014,0,1.0,0.0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,31.0,30,9,2014,30,9,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
62092,-1.0,30,9,2014,30,9,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
62093,-1.0,30,9,2014,30,9,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
62094,-1.0,30,9,2014,30,9,2014,0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0


In [ ]:
# Get the probabilities of the test dataset
test_users = pd.read_csv('/content/drive/MyDrive/data bigdata/test_users.csv')

test_df = test_users[['id']]
test_ids = test_df['id']
y_test_prob = model1.predict_proba(train_df)

# Extract the top 5 countries for each member and create a DataFrame
ids = []
countries = []

for i, member_id in enumerate(test_ids):
    ids.extend([member_id] * 5)
    top_5_indices = np.argsort(y_test_prob[i, :])[::-1][:5]
    top_5_countries = [label_encoder.classes_[index] for index in top_5_indices]
    countries.extend(top_5_countries)

# Create a DataFrame 'MLpred' from 'ids' and 'countries'
model1_pd = pd.DataFrame({'id': ids, 'country': countries})

In [ ]:
test_users

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE
62092,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,NaN,basic,23,ko,direct,direct,untracked,Android,Android Phone,-unknown-
62093,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,NaN,basic,0,de,direct,direct,linked,Web,Windows Desktop,Firefox
62094,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari


In [ ]:
model1_pd

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,IT
...,...,...
310475,9uqfg8txu3,NDF
310476,9uqfg8txu3,US
310477,9uqfg8txu3,other
310478,9uqfg8txu3,FR


In [ ]:
merged_model1_df = pd.merge(test_users, model1_pd, on='id', how='inner')
merged_model1_df

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,NDF
1,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,US
2,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,other
3,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,FR
4,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310475,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,NDF
310476,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,US
310477,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,other
310478,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,FR


In [ ]:
#merged_df.to_csv('predictionModel1.csv',index=False)

##Model2

In [ ]:
from xgboost import XGBClassifier
# Initialize an estimator with the best param set from the first tuning session
gb_best1 = XGBClassifier(objective='multi:softprob',eval_metric= 'ndcg',
      learning_rate=0.1,
      max_depth=3,
      min_child_weight=0.5,
      n_estimators=100,
      n_jobs=4,
      random_state=42)

# Fit on the whole training dataset
gb_best1.fit(train_df, encoded_y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='ndcg', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=0.5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=4,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# Calculating NDCG Score on the training set
best1_score = ndcg_score_udf(encoded_y_train, gb_best1.predict_proba(train_df))
print(f"NDCG Training Score on the first best estimator: {best1_score}")

# Extracting feature importance
feat_importance = gb_best1.feature_importances_

# Getting the sorted indices to implement feature selection later
sorted_indices = np.argsort(feat_importance)

# Getting the index of the first feature with importance less than a threshold (for example 0.001)
print(f"The index of the first feature with importance less than 0.001 is: {np.where(np.array(sorted(feat_importance, reverse=True)) < 0.001)[0][0]}")

NDCG Training Score on the first best estimator: 0.8330441833908524
The index of the first feature with importance less than 0.001 is: 171


In [ ]:
# Get the probabilities of the test dataset
test_ids = test_df['id']
y_test_prob = gb_best1.predict_proba(train_df)

# Extract the top 5 countries for each member and create a DataFrame
ids = []
countries = []

for i, member_id in enumerate(test_ids):
    ids.extend([member_id] * 5)
    top_5_indices = np.argsort(y_test_prob[i, :])[::-1][:5]
    top_5_countries = [label_encoder.classes_[index] for index in top_5_indices]
    countries.extend(top_5_countries)

# Create a DataFrame 'MLpred' from 'ids' and 'countries'
MLpred = pd.DataFrame({'id': ids, 'country': countries})

In [ ]:
MLpred = pd.merge(test_users, MLpred, on='id', how='inner')
MLpred

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,NDF
1,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,US
2,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,FR
3,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,other
4,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310475,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,NDF
310476,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,US
310477,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,other
310478,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49.0,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome,FR


In [ ]:
#MLpred.to_csv('predModel2-Classificat.csv',index=False)